# Create a View of Source Table `[Uploadfile].[CurrentSemesterCourseOfferings]`


In [39]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create View `[Uploadfile].[vw_CurrentSemesterCourseOfferings]` from source table `[Uploadfile].[CurrentSemesterCourseOfferings]`. 
--                ChatGPT 4o assisted >
-- =============================================

USE QueensClassSchedule
GO

CREATE VIEW [Uploadfile].[vw_CurrentSemesterCourseOfferings] AS
SELECT
    [Semester],
    [Sec],
    [Code],
    [Course (hr, crd)] AS OriginalCourseColumn,
    -- Extract CourseName (text before the first '(')
    LTRIM(RTRIM(LEFT([Course (hr, crd)], CHARINDEX('(', [Course (hr, crd)]) - 1))) AS CourseName,
    -- Extract hr (number between '(' and ',')
    LTRIM(RTRIM(SUBSTRING([Course (hr, crd)],
        CHARINDEX('(', [Course (hr, crd)]) + 1,
        CHARINDEX(',', [Course (hr, crd)]) - CHARINDEX('(', [Course (hr, crd)]) - 1))) AS hr,
    -- Extract crd (number between ',' and ')')
    LTRIM(RTRIM(SUBSTRING([Course (hr, crd)],
        CHARINDEX(',', [Course (hr, crd)]) + 1,
        CHARINDEX(')', [Course (hr, crd)]) - CHARINDEX(',', [Course (hr, crd)]) - 1))) AS crd,
    [Description],
    [Day],
    [Time],
    [Instructor],
    [Location],
    [Enrolled],
    [Limit],
    [Mode of Instruction]
FROM
    [Uploadfile].[CurrentSemesterCourseOfferings];


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.013

# Create the required schemas and User-defined Data Types

### Schemas being created:
- `[Udt]`
- `[DbSecurity]`
- `[Course]`
- `[Location]`
- `[Department]`
- `[Process]`
- `[Project3]`

In [40]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski, Jascharan Singh, Sarmad Ali, Augusta Na, Carlos Vega >
-- Create date: < 12/3/2024 >
-- Description:	< Create the required schemas and UDT's >
-- =============================================

USE QueensClassSchedule
GO

CREATE  SCHEMA  [Udt]
GO

CREATE  TYPE    [Udt].[SurrogateKeyInt]     FROM    [int]               NULL
GO
CREATE  TYPE    [Udt].[DateAdded]           FROM    Datetime2(7)    NOT NULL
GO
CREATE  TYPE    [Udt].[ClassTime]           FROM    NCHAR(5)        NOT NULL
GO
CREATE  TYPE    [Udt].[IndividualProject]   FROM    NVARCHAR(60)    NOT NULL
GO
CREATE  TYPE    [Udt].[LastName]            FROM    NVARCHAR(35)    NOT NULL
GO
CREATE  TYPE    [Udt].[FirstName]           FROM    NVARCHAR(20)    NOT NULL
GO
CREATE  TYPE    [Udt].[GroupName]           FROM    NVARCHAR(20)    NOT NULL
GO
CREATE  TYPE    [Udt].[String100]           FROM    NVARCHAR(100)   NOT NULL
GO
CREATE  TYPE    [Udt].[Rows]                FROM    [int]           NOT NULL
GO
CREATE  TYPE    [Udt].[String4]             FROM    NVARCHAR(4)         NULL
GO
CREATE  TYPE    [Udt].[String8]             FROM    NVARCHAR(8)         NULL
GO
CREATE  TYPE    [Udt].[String20]            FROM    NVARCHAR(20)        NULL
GO
CREATE  TYPE    [Udt].[String30]            FROM    NVARCHAR(30)    NOT NULL
GO
CREATE  TYPE    [Udt].[ClassLength]         FROM    NVARCHAR(30)    NOT NULL
GO
CREATE  TYPE    [Udt].[FloatField]          FROM    FLOAT           NOT NULL
GO

-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create the required schemas >
-- =============================================

USE QueensClassSchedule
GO

CREATE  SCHEMA  [DbSecurity]
GO
CREATE  SCHEMA  [Course]
GO
CREATE  SCHEMA  [Location]
GO
CREATE  SCHEMA  [Department]
GO
CREATE  SCHEMA  [Process]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.066

# Create `[DbSecurity].[UserAuthorization]` Table

### Create and populate the table storing the group members:

In [41]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create the `[DbSecurity].[UserAuthorization]` table >
-- =============================================

USE QueensClassSchedule
GO

CREATE  TABLE   [DbSecurity].[UserAuthorization]    (
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [ClassTime]             [Udt].[ClassTime]                           NULL    DEFAULT ('10:45'),
    [IndividualProject]     [Udt].[IndividualProject]                   NULL    DEFAULT ('Project 3'),
    [GroupMemberLastName]   [Udt].[LastName]                        NOT NULL,
    [GroupMemberFirstName]  [Udt].[FirstName]                       NOT NULL,
    [GroupName]             [Udt].[GroupName]                       NOT NULL    DEFAULT ('Group 2'),
    [DateAdded]             [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME())
);
GO

INSERT  INTO    [DbSecurity].[UserAuthorization]    (GroupMemberLastName,   GroupMemberFirstName)
VALUES
    ('Sarmad',      'Ali'), 
    ('Daniel',      'Gargiullo'), 
    ('Augusta',     'Na'), 
    ('Jascharan',   'Singh'),
    ('Carlos',      'Vega Lemus'), 
    ('Oksana',      'Weigand-Suminski');

Commands completed successfully.

Commands completed successfully.

(6 rows affected)

Total execution time: 00:00:00.014

# Create Required Tables
- `[Process]`
    - `.[WorkFlowSteps]`
- `[Course]`
    - `.[Course]`
    - `.[Class]`
    - `.[ModeOfInstruction]`
- `[Location]`
    - `.[BuildingLocation]`
    - `.[RoomLocation]`
- `[Department]`
    - `.[Department]`
    - `.[Instructor]`

In [42]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create the other required tables >
-- =============================================

USE QueensClassSchedule
GO

/* [Process] schema */

CREATE  TABLE   [Process].[WorkFlowSteps]   (
    [WorkFlowStepsKey]              [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [WorkFlowStepsDescription]      [Udt].[String100]                       NOT NULL,
    [WorkFlowStepsTableRowCount]    [Udt].[Rows]                                NULL    DEFAULT ((0)),
    [StartingDateTime]              [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME()),
    [EndDateTime]                   [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME()),
    [ClassTime]                     [Udt].[ClassTime]                           NULL    DEFAULT ('10:45'),

    [UserAuthorizationKey]          [Udt].[SurrogateKeyInt]                 NOT NULL
)
GO

/* [Course] schema */

CREATE  TABLE   [Course].[Course]   (
	[CourseID]              [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
	[CourseCode]            [Udt].[String30]                            NULL,
	[Description]           [Udt].[String100]                           NULL,
    [Section]               [Udt].[String4]                             NULL,
    [Semester]              [Udt].[String20]                            NULL,
    -- Required fields:
	[UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
	[DateAdded]             [Udt].[DateAdded]                           NULL,
	[DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

CREATE  TABLE   [Course].[Class]    (
    [ClassID]               [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [Enrollment]            [Udt].[SurrogateKeyInt]                     NULL,
    [Limit]                 [Udt].[SurrogateKeyInt]                     NULL,
    [Section]               [Udt].[String30]                            NULL,
    [ClassCode]             [Udt].[String30]                            NULL,
    [Days]                  [Udt].[String30]                            NULL,
    [Time]                  [Udt].[ClassLength]                         NULL,
    [ClassStart]            [Udt].[String20]                           NULL,
    [ClassEnd]              [Udt].[String20]                           NULL,
    [Hours]                 [Udt].[FloatField]                          NULL,
    [Credits]               [Udt].[SurrogateKeyInt]                     NULL,
    [InstructorID]          [Udt].[SurrogateKeyInt]                     NULL,
    -- Required fields:
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
    [DateAdded]             [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME()),
    [DateOfLastUpdate]      [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME())
)
GO

CREATE  TABLE   [Course].[ModeOfInstruction]    (
	[ModeID]                [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
	[ModeName]              [Udt].[String100]                       NOT NULL,
    -- Required fields:
	[UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
	[DateAdded]             [Udt].[DateAdded]                           NULL,
	[DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
);

-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski, Carlos Vega >
-- Create date: < 12/5/2024 >
-- Description:	< Create the required tables in the `[Location]` schema >
-- =============================================

/* [Location] schema */

CREATE  TABLE   [Location].[BuildingLocation]   (
    [BuildingLocationID]    [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL 
        PRIMARY KEY CLUSTERED,
    [CourseCode]            [Udt].[String30]                        NOT NULL,
    [Description]           [Udt].[String100]                           NULL,
    [Building]              [Udt].[String30]                        NOT NULL,
    [Day]                   [Udt].[String30]                            NULL,
    -- Required fields:
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
    [DateAdded]             [Udt].[DateAdded]                           NULL,
    [DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

CREATE  TABLE   [Location].[RoomLocation]   (
	[RoomLocationID]        [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [Room]                  [Udt].[String30]                        NOT NULL,
    [Time]                  [Udt].[String20]                        NOT NULL,
    [Day]                   [Udt].[String20]                        NOT NULL,
	[BuildingLocationID]    [Udt].[SurrogateKeyInt]                 NOT NULL,
    -- Required fields:
	[UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
	[DateAdded]             [Udt].[DateAdded]                           NULL,
	[DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

/* [Department] schema */

CREATE  TABLE   [Department].[Department]   (
    [DepartmentID]          [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [InstructorID]          [Udt].[SurrogateKeyInt]                 NOT NULL,
    [DepartmentName]        [Udt].[String8]                         NOT NULL    UNIQUE,
    -- Required fields:
	[UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
	[DateAdded]             [Udt].[DateAdded]                           NULL,
	[DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

/***** Possibly implementing this or something similar: *****/
-- Insert department names into the Department table
-- INSERT INTO [Department].[Department] (DepartmentName)
-- SELECT DISTINCT
--     LEFT([Course (hr, crd)], CHARINDEX(' ', [Course (hr, crd)]) - 1)
-- FROM [Uploadfile].[CurrentSemesterCourseOfferings]
-- WHERE [Course (hr, crd)] IS NOT NULL AND CHARINDEX(' ', [Course (hr, crd)]) > 0;
-- GO
/************************************************************/

CREATE  TABLE   [Department].[Instructor]   (
    [InstructorID]          [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [DepartmentID]          [Udt].[SurrogateKeyInt]                 NOT NULL,
    [FirstName]             [Udt].[String30]                        NOT NULL,
    [LastName]              [Udt].[String30]                        NOT NULL,
    -- Required fields:
	[UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
	[DateAdded]             [Udt].[DateAdded]                           NULL,
	[DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.033

# Create Workflow Procedures

- `[Process]`
    - `[usp_TrackWorkFlow]`
    - `[usp_ShowWorkFlowSteps]`

In [43]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/5/2024 >
-- Description:	< Create `[Process].[usp_TrackWorkFlow]` and `[Process].[usp_ShowWorkFlowSteps]` procedures >
-- =============================================

USE QueensClassSchedule
GO

CREATE PROCEDURE [Process].[usp_TrackWorkFlow]
    @UserAuthorizationKey INT,
    @WorkFlowStepsTableRowCount INT,
    @WorkFlowStepsDescription NVARCHAR(100)
AS
BEGIN
    INSERT INTO [Process].[WorkFlowSteps]
        (UserAuthorizationKey, WorkFlowStepsTableRowCount, WorkFlowStepsDescription)
    VALUES(@UserAuthorizationKey, @WorkFlowStepsTableRowCount, @WorkFlowStepsDescription);
END;
GO

CREATE PROCEDURE [Process].[usp_ShowWorkFlowSteps] 
AS
BEGIN
	SELECT *
	FROM [Process].[WorkFlowSteps];
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.013

In [44]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/6/2024 >
-- Description:	< Create the `[Process].[usp_TrackWorkFlowSteps]` >
-- =============================================

USE QueensClassSchedule
GO

CREATE PROCEDURE [Course].[usp_LoadClassTable]
AS
BEGIN
    -- Populate the Course.Class table with start and end times split from the Time column
    INSERT INTO [Course].[Class] (
        Section,
        ClassCode,
        Days,
        Time,
        ClassStart,
        ClassEnd,
        Hours,
        Credits,
        UserAuthorizationKey,
        DateAdded,
        DateOfLastUpdate
    )
    SELECT
        [Sec] AS Section,
        LEFT([Course (hr, crd)], CHARINDEX(' ', [Course (hr, crd)]) - 1) AS ClassCode, -- Extract the department code
        [Day] AS Days,
        [Time] AS Time,
        -- Extract ClassStart and ClassEnd from the Time column
        LTRIM(RTRIM(LEFT([Time], CHARINDEX('-', [Time]) - 1))) AS ClassStart,
        LTRIM(RTRIM(SUBSTRING([Time], CHARINDEX('-', [Time]) + 1, LEN([Time])))) AS ClassEnd,
        TRY_CAST(LTRIM(RTRIM(SUBSTRING([Course (hr, crd)],
            CHARINDEX('(', [Course (hr, crd)]) + 1,
            CHARINDEX(',', [Course (hr, crd)]) - CHARINDEX('(', [Course (hr, crd)]) - 1))) AS FLOAT) AS Hours,
        TRY_CAST(LTRIM(RTRIM(SUBSTRING([Course (hr, crd)],
            CHARINDEX(',', [Course (hr, crd)]) + 1,
            CHARINDEX(')', [Course (hr, crd)]) - CHARINDEX(',', [Course (hr, crd)]) - 1))) AS INT) AS Credits,
        NULL AS UserAuthorizationKey, -- Provide appropriate key if required
        GETDATE() AS DateAdded,
        GETDATE() AS DateOfLastUpdate
    FROM
        [Uploadfile].[CurrentSemesterCourseOfferings]
    WHERE
        [Course (hr, crd)] IS NOT NULL
        AND CHARINDEX(' ', [Course (hr, crd)]) > 0
        AND CHARINDEX('-', [Time]) > 0; -- Ensure Time column contains a valid range

         --  Insert the user into the Process.WorkFlowTable
    EXEC [Course].[TrackWorkFlow] @UserAuthorizationKey = @UserAuthorizationKey, @WorkflowStepsDescription =  'Loading data into Class table',
    @WorkflowStepsTableRowCount = @@ROWCOUNT;
END;
GO


Commands completed successfully.

: Msg 137, Level 15, State 2, Procedure usp_LoadClassTable, Line 44
Must declare the scalar variable "@UserAuthorizationKey".

Total execution time: 00:00:00.002